# Exscientia 2023.4.0 Demo

In [ ]:
import sire as sr

Central to the free energy functionality is the concept of a merged molecule

In [ ]:
mols = sr.load(sr.expand(sr.tutorial_url, "merged_molecule.s3"))

This is a molecule for which `is_perturbable` is `True`, so can be easily extracted from the other molecules.

In [ ]:
mol = mols["molecule property is_perturbable"]
mol

Merged molecules have two sets of properties. The `xxx0` properties are for the reference state, while the `xxx1` properties are for the perturbed state.

In [ ]:
mol.property("charge0")

In [ ]:
mol.property("charge1")

We need to choose which of these two states should be the "default" view of the molecule. Here we will choose the reference state, which we will view.

In [ ]:
mol = mol.perturbation().link_to_reference().commit()
mol.view()

Here we choose the perturbed state as the default, and view all of the non-ghost atoms.

In [ ]:
mol.perturbation().link_to_perturbed().commit()["not element Xx"].view()

We also provide extra functions in `sire.morph` that are useful for free energy calculations. For example, we have `sire.morph.repartition_hydrogen_masses()` which exposes the functionality built for BioSimSpace in sire.

In [ ]:
mol.property("mass")

In [ ]:
mol = sr.morph.repartition_hydrogen_masses(mol)
mol.property("mass")

Now we've updated the molecule, let's minimise it at λ=0.5, applying no constraints to the merged molecule, but constraint the `h-bonds` for all other molecules.

In [ ]:
mols.update(mol)
mols = mols.minimisation(lambda_value=0.5, 
                         constraint="h-bonds", 
                         perturbable_constraint="none").run().commit()

We are now ready to run dynamics at the same λ-value using the same constraints.

In [ ]:
d = mols.dynamics(timestep="4fs", 
                  temperature="25oC", 
                  constraint="h-bonds",
                  perturbable_constraint="none",
                  lambda_value=0.5)}

Let's run 10 ps of dynamics, collecting energies every 0.5 ps

In [ ]:
d.run("10ps", energy_frequency="0.5ps")

The energy trajectory is available via the `energy_trajectory` function. Unsurprisingly, it is an `EnergyTrajectory` object.

In [ ]:
d.energy_trajectory()

Let's run another 10 ps of dynamics, but this time let's also calculate the energies at λ=0 and λ=1 for every step along the trajectory at λ=0.5

In [ ]:
d.run("10ps", energy_frequency="0.5ps", lambda_windows=[0.0, 1.0])

In [ ]:
e = d.energy_trajectory()
e

It is easier to see the trajectory as a pandas DataFrame.

In [ ]:
e.to_pandas()

Even better, let's use a pandas DataFrame that has been formatted to be compatible with alchemlyb :-)

In [ ]:
a = e.to_alchemlyb()
a

In [ ]:
a.attrs

We can calculate relative free energies lots of different methods. The simplest would be just running dynamics at different λ-values, and calculating the energy to neighbouring λ-values. There's example of such a script in [the tutorial](https://sire.openbiosim.org/tutorial/part06/06_faster_fep.html#complete-example-script).

In good Blue Peter style, I've run it on my computer so here's on I made earlier ;-)]

In [ ]:
!ls bluepeter

In this case, we've streamed the `EnergyTrajectory` objects and saved them in the sire saved stream format (.s3, also the same as .bss). This is a compact, compressed, binary format that perfectly encodes all of the information about the energy trajectory, including the λ-values, temperature, energy units etc.

The `sire.morph.to_alchemlyb` function can convert a whole set of `EnergyTrajectory` objects (reading them from disk if needed) into a single alchemlyb-compatible DataFrame. This does the work needed to arrange the `EnergyTrajectory` objects from each λ-value into the right order so that the right free energy is calculated.

In [ ]:
de_water = sr.morph.to_alchemlyb("bluepeter/energy_water_*")
de_water

In [ ]:
de_water.attrs

This is now ready for free energy estimation using the BAR method (using BAR as, in this case, we only have the energies for neighbouring λ-windows)

In [ ]:
from alchemlyb.estimators import BAR

In [ ]:
bar = BAR()
bar.fit(de_water)
dG_water = bar.delta_f_.loc[0.00, 1.00]
dG_water

Let's do the same for the vacuum leg...

In [ ]:
de_vac = sr.morph.to_alchemlyb("bluepeter/energy_vacuum_*")
bar = BAR()
bar.fit(de_vac)
dG_vac = bar.delta_f_.loc[0.00, 1.00]
dG_vac

The relative hydration free energy of ethane and methanol is...

In [ ]:
dG_water - dG_vac

We are compatible with all of the functionality in alchemlyb, e.g. here we pass in our s3 files to calculate and plot convergence.

In [ ]:
from glob import glob
from alchemlyb.convergence import forward_backward_convergence
from alchemlyb.visualisation import plot_convergence
dfs = []
s3files = glob("bluepeter/energy_water*.s3")
s3files.sort()
for s3 in s3files:
   dfs.append(sr.stream.load(s3).to_alchemlyb())
f = forward_backward_convergence(dfs, "bar")
plot_convergence(f)

Under the hood, this is all based on STANDARD OpenMM. The `mols.dynamics()` function just simplifies the function calls that are needed to create the OpenMM context. The context is seen hidden in the `Dynamics` object...

In [ ]:
d = mols.dynamics(timestep="4fs", temperature="25oC", lambda_value=0.5)
d

In [ ]:
d._d._omm_mols

We could have created this ourselves directly using the `sr.convert` function.

In [ ]:
c = sr.convert.to(mols, "openmm", map={"timestep": sr.u("4fs"),
                                       "temperature": sr.u("25oC"),
                                       "lambda_value": 0.5,
                                       "constraint": "h-bonds",
                                       "perturbable_constraint": "none"})
c

We have created a small overload of the openmm.Context class that holds additional metadata and provides some convenience functions for updating the parameters in the forces for the different values of λ.

In [ ]:
c.__class__, c.__class__.__bases__

In [ ]:
help(c)

All the existing OpenMM functions are the same, e.g. you can still get and use the System and Integrator as before...

In [ ]:
c.getSystem()

In [ ]:
c.getIntegrator()

But now we have some additional pure python functions that let you easily set the parameters for different λ-values and ask OpenMM to calculate energies.

In [ ]:
c.set_lambda(0.2)

In [ ]:
c.get_potential_energy()

In [ ]:
for l in range(0, 11):
    lam = l / 10.0
    c.set_lambda(lam)
    print(lam, c.get_potential_energy())

In [ ]:
integrator = c.getIntegrator()
integrator.step(100)
print(c.get_potential_energy())
c.set_lambda(0.5)
print(c.get_potential_energy())

Perturbations are handled via almost completely standard OpenMM Forces.

In [ ]:
c.getSystem().getForces()

This means that we are compatible with lots of cool functionality in OpenMM, and we directly support PME without doing any additional work :-)

Perturbations are handled via a `LambdaLever` class which stores the parameters for the molecules and can interpolate those parameters for different values of λ.

The `LambdaLever` then calls `updateParametersInContext` for all the forces, thus updating the parameters in the OpenMM Context for the desired λ-value. This means that λ DOES NOT exist in the OpenMM Context, and there is NO slowdown caused by use of custom forcefields that try to perturb parameters by λ directly within the integrator.

A further benefit is that we have a huge amount of flexibility over how we perturb parameters with λ.

This is controlled using a `LambdaSchedule`. This class describes the schedule by which parameters will be morphed as a function of λ.

In [ ]:
l = sr.cas.LambdaSchedule()
l

First, we will add a simple `morph` stage, which just morphs parameters using the simple morphing equationl

In [ ]:
l.add_stage("morph", (1-l.lam()) * l.initial() + l.lam() * l.final())
l

This means that all perturbable parameters will be morped from their initial to final values using the above equation.

But, lets change the charges by a different equation...

In [ ]:
l.set_equation("morph", "charge", 0.2 * ((1-l.lam()) * l.initial() + l.lam() * l.final()))
l

We can plot the effect of the lever on some hyperthetical parameters that go from 1 to 2

In [ ]:
l.get_lever_values(initial=1.0, final=2.0).plot()

And maybe we want to scale the LJ epsilon parameter by λ squared...

In [ ]:
l.set_equation("morph", "epsilon", (1-l.lam()**2) * l.initial() + l.lam()**2 * l.final())
l.get_lever_values(initial=1.0, final=2.0).plot()

This is super powerful. For example, here is how you would create a `LambdaSchedule` that scales down the charges before performing the morph, and then scales the charges back to normal.

In [ ]:
l = sr.cas.LambdaSchedule()
l.add_morph_stage("morph")
l.add_charge_scale_stages("decharge", "recharge", scale=0.2)
l.get_lever_values(initial=1.0, final=2.0).plot()

Or here is the schedule for turning on restraints that are held on during the morph, and then switched off afterwards...

In [ ]:
l = sr.cas.LambdaSchedule()
l.add_stage("restraints_on", l.initial())
l.add_stage("morph", (1-l.lam()) * l.initial() + l.lam() * l.final())
l.add_stage("restraints_off", l.final())
l.set_equation("restraints_on", "restraint", l.lam() * l.initial())
l.set_equation("restraints_off", "restraint", (1-l.lam()) * l.initial())
l.set_equation("morph", "restraint", l.initial())

l.get_lever_values(initial=1.0, final=2.0).plot()

Our current implementation supports positional restraints, distance (bond) restraints, and a basic implementation of Boresch restraints.

We also now support fixing atoms, e.g. you can freeze all atoms outside a radius from the ligand, and use a positional half-harmonic restraint to hold all other atoms within a sphere centered on the ligand.

This is all now available in `sire 2023.4.0`, with functionality fully described in [the tutorial](https://sire.openbiosim.org/tutorial/index_part06.html).